# Crop Raster to 1024*1024 for Waste identification

In [1]:
import rasterio
from rasterio.windows import Window
from PIL import Image
import os
import csv

In [2]:
# Define the function to cut the raster into 1024x1024 tiles, convert to JPG with worldfile, and record metadata in CSV
def cut_raster_to_jpg_with_csv(input_raster, output_folder, city, tile_size=1024):
    # Prepare the output CSV file
    csv_file = os.path.join(output_folder, f'{city}_metadata.csv')
    csv_fields = ['img_id', 'city', 'tile_row', 'tile_col', 'tile_width', 'tile_height', 
                  'x_coord', 'y_coord', 'center_point', 'grid_polygon']

    # Create the CSV file and write the header
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_fields)
        writer.writeheader()

        with rasterio.open(input_raster) as src:
            width = src.width
            height = src.height

            # Calculate number of tiles in both dimensions
            num_tiles_x = width // tile_size
            num_tiles_y = height // tile_size

            for i in range(num_tiles_y):
                for j in range(num_tiles_x):
                    # Define window for the tile
                    window = Window(j * tile_size, i * tile_size, tile_size, tile_size)

                    # Read the tile from the raster
                    tile = src.read(window=window)

                    # Save the image as JPG using PIL
                    img_id = f'{city}_tile_{i}_{j}'
                    output_jpg = os.path.join(output_folder, f'{img_id}.jpg')
                    img = Image.fromarray(tile.transpose(1, 2, 0))
                    img.save(output_jpg)

                    # print(f"Image saved as {output_jpg}")

                    # # Create world file (.jgw) for georeferencing (commented out as per request)
                    # world_file = os.path.splitext(output_jpg)[0] + '.jgw'
                    # with open(world_file, 'w') as f:
                    #     f.write(f'{src.transform.a}\n')  # Pixel size in the X direction
                    #     f.write(f'{src.transform.b}\n')  # Rotation (0 if north is up)
                    #     f.write(f'{src.transform.d}\n')  # Rotation (0 if north is up)
                    #     f.write(f'{src.transform.e}\n')  # Pixel size in the Y direction (negative value)
                    #     f.write(f'{src.transform.c + j * tile_size * src.transform.a}\n')  # X coord of the upper-left
                    #     f.write(f'{src.transform.f + i * tile_size * src.transform.e}\n')  # Y coord of the upper-left

                    # Calculate the bounding box coordinates
                    min_x = src.transform.c + j * tile_size * src.transform.a
                    max_x = min_x + tile_size * src.transform.a
                    max_y = src.transform.f + i * tile_size * src.transform.e
                    min_y = max_y + tile_size * src.transform.e

                    # Calculate center point coordinates
                    center_x = (min_x + max_x) / 2
                    center_y = (min_y + max_y) / 2

                    # Create grid polygon in WKT format
                    grid_polygon = f'POLYGON(({min_x} {min_y}, {min_x} {max_y}, {max_x} {max_y}, {max_x} {min_y}, {min_x} {min_y}))'

                    # Record metadata in the CSV
                    metadata = {
                        'img_id': img_id,
                        'city': city,
                        'tile_row': i,
                        'tile_col': j,
                        'tile_width': tile.shape[2],  # Width of the tile
                        'tile_height': tile.shape[1],  # Height of the tile
                        'x_coord': min_x,
                        'y_coord': max_y,
                        'center_point': f'POINT({center_x} {center_y})',
                        'grid_polygon': grid_polygon
                    }
                    writer.writerow(metadata)

        print(f"Cutting, conversion, and CSV metadata recording complete! Files saved to {output_folder}")

In [3]:
# Example usage
input_raster = '/Users/wenlanzhang/Downloads/15_13722_2676_大图/L19/15_13722_2676.tif'  # Input raster file
output_folder = '/Users/wenlanzhang/Downloads/15_13722_2676_大图'  # Output folder
city = 'BJ'  # City name
cut_raster_to_jpg_with_csv(input_raster, output_folder, city)

Cutting, conversion, and CSV metadata recording complete! Files saved to /Users/wenlanzhang/Downloads/15_13722_2676_大图
